In [ ]:
#this is the code for identifying  keystone species

#question 1 for implentation: How is the proposer going to work?

explore using mult-arm bandits or hyperparameter optimization

bayesian information criteria is the reward. the accuracy aspect of BIC will be defined as how close to the data it is. 

the data used will be MEG or EEG. 

the policies will be trained normally. number of groupings of agents will be a hyperparameter. these instances will be trained in parallel fashion

centralized learning will be used. 

This is because 

the number of groupings will increase until and be compared to a smaller number of groupings until reward stops improving. 

libraries used:

agilerl


In [1]:
from ray.rllib.connectors.env_to_module import FlattenObservations
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.examples.envs.classes.two_step_game import TwoStepGameWithGroupedAgents
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import register_env, get_trainable_cls

pygame 2.5.1 (SDL 2.28.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import torch
import numpy as np
import sys, copy, math, time, pdb
import os.path
import random
import pdb
import csv
import argparse
import itertools
from itertools import permutations, product
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchdiffeq import odeint
import itertools

import torch
from pettingzoo.mpe import simple_speaker_listener_v4
from tqdm import trange

from agilerl.components.multi_agent_replay_buffer import MultiAgentReplayBuffer
from agilerl.wrappers.pettingzoo_wrappers import PettingZooVectorizationParallelWrapper

from agilerl.algorithms.matd3 import MATD3

https://github.com/spxuw/DKI/blob/main/DKI.py

#import libraries 

In [2]:
import numpy as np
from gymnasium.spaces import Dict, Discrete
import argparse
import os

from ray import air, tune
from ray.air.constants import TRAINING_ITERATION
#from ray import rllib

each time the algorithm decides it a new skill will be created 